In [1]:
from bs4 import BeautifulSoup
import requests

from scipy import stats
import pandas as pd

from random import random
from time import sleep
from time import time
from IPython.core.display import clear_output

from itertools import cycle

In [2]:
#Set the folder for APIKEY.py file
%cd "/nfshome/pmb434"

#Load API Keys
from APIKEY import ProxyMeshUser
from APIKEY import ProxyMeshPass

#Go back to previuos folder
%cd -

/nfshome/pmb434
/projects/ml_final_project/shared


In [3]:
dob_now = pd.read_csv('data/DOB_NOW__Safety_Boiler.csv')
dob_now.head()

,Tracking Number,Boiler ID,Report Type,ApplicantFirst Name,Applicant Last Name,Applicant License Type,Applicant License Number,Owner First Name,Owner Last Name,Boiler Make,...,Pressure Type,Inspection Type,Inspection Date,Defects Exist,LFF (45 Days),LFF (180 Days),Filing Fee,Total Amount Paid,Report Status,Bin Number
0,2017-10000008199Y0001-24915,10000008199Y0001,Initial,DREW,THOMAS,BB,5468.0,BARBARA,KAMMERER,ROCKMILLS STE,...,Low Pressure,External,11/17/2017 00:00:00,No,0,0,30,30,Accepted,1059945
1,2017-10000017650Y0001-24577,10000017650Y0001,Initial,BENJAMIN,WAGNER,BB,5469.0,LINDA,BALIDEMAJ,ROCKMILLS,...,Low Pressure,External,11/15/2017 00:00:00,No,0,0,30,30,Accepted,1017834
2,2017-10000017652Y0001-24652,10000017652Y0001,Initial,JOHN,KNIEF,O,5317.0,CHRISTINA,RIVERA,H.B. SMITH,...,Low Pressure,External,11/14/2017 00:00:00,No,0,0,30,30,Accepted,1016116
3,2017-10000018251Y0001-24562,10000018251Y0001,Initial,BENJAMIN,WAGNER,BB,5469.0,NATE,KLEIN,PEERLESS,...,Low Pressure,External,11/15/2017 00:00:00,No,0,0,30,30,Accepted,1006432
4,2017-10000018279Y0001-24631,10000018279Y0001,Initial,EDWIN,VAZQUEZ,P,1526.0,LEONARD,EIS,CROTTY,...,Low Pressure,External,11/17/2017 00:00:00,No,0,0,30,30,Accepted,1006285


### Prepare boiler data

In [4]:
boilers = dob_now[['Boiler ID', 'Bin Number']].drop_duplicates('Boiler ID')

In [5]:
boilers['boro'] = boilers['Boiler ID'].apply(lambda x: str(x)[0])

In [6]:
boilers.boro.value_counts()

3    31285
1    29927
4    20100
2    12403
5     2013
0        5
B        2
n        2
Name: boro, dtype: int64

In [7]:
boilers = boilers.loc[(boilers.boro != 'n') & (boilers.boro != 'B')]
boilers.boro = boilers.boro.astype(int)
boilers = boilers.loc[(boilers.boro > 0)]
boilers.boro.value_counts()

3    31285
1    29927
4    20100
2    12403
5     2013
Name: boro, dtype: int64

In [8]:
boilers['short_id'] = pd.to_numeric(boilers['Boiler ID'].apply(lambda x: x[1:11]), errors='coerce')
boilers.dropna(inplace=True)
boilers.short_id = boilers.short_id.astype(int)
boilers.head()

,Boiler ID,Bin Number,boro,short_id
0,10000008199Y0001,1059945,1,8199
1,10000017650Y0001,1017834,1,17650
2,10000017652Y0001,1016116,1,17652
3,10000018251Y0001,1006432,1,18251
4,10000018279Y0001,1006285,1,18279


In [9]:
boilers['serial'] = pd.to_numeric(boilers['Boiler ID'].apply(lambda x: x[12:]), errors='coerce')
boilers.dropna(inplace=True)
boilers.serial = boilers.serial.astype(int)
boilers.head()

,Boiler ID,Bin Number,boro,short_id,serial
0,10000008199Y0001,1059945,1,8199,1
1,10000017650Y0001,1017834,1,17650,1
2,10000017652Y0001,1016116,1,17652,1
3,10000018251Y0001,1006432,1,18251,1
4,10000018279Y0001,1006285,1,18279,1


In [10]:
(boilers.serial > 1).sum()

21933

In [11]:
boilers[boilers.duplicated(subset=['boro','serial'], keep=False)]

,Boiler ID,Bin Number,boro,short_id,serial
0,10000008199Y0001,1059945,1,8199,1
1,10000017650Y0001,1017834,1,17650,1
2,10000017652Y0001,1016116,1,17652,1
3,10000018251Y0001,1006432,1,18251,1
4,10000018279Y0001,1006285,1,18279,1
5,10000018853Y0001,1048707,1,18853,1
6,10000002683N0001,1015989,1,2683,1
7,10000015933Y0001,1063409,1,15933,1
8,10000023237Y0001,1003771,1,23237,1
9,10000027669Y0001,1064737,1,27669,1


In [12]:
len(boilers)

95708

### Try scraping one boiler

In [188]:
url = 'http://a810-bisweb.nyc.gov/bisweb/BoilerComplianceQueryServlet?allborough={}&boilernumber={}'.format(1, 18279)
response = requests.get(url)

In [189]:
html_soup = BeautifulSoup(response.text, 'html.parser')

In [190]:
table = html_soup.find_all('table')[3]

In [191]:
link = table.find_all('a')[0]
link.get('href')

'BoilerComplianceDisplayServlet?requestid=0&allisn=0000120382&allbin='

In [192]:
url2 = 'http://a810-bisweb.nyc.gov/bisweb/' + link.get('href')
url2

'http://a810-bisweb.nyc.gov/bisweb/BoilerComplianceDisplayServlet?requestid=0&allisn=0000120382&allbin='

In [193]:
response2 = requests.get(url2)
html_soup2 = BeautifulSoup(response2.text, 'html.parser')

In [194]:
table2 = html_soup2.find_all('table')

In [195]:
data_table = table2[6]

In [ ]:
html_soup2.find(text='')

In [196]:
data = pd.read_html(str(data_table), skiprows=4, header=0)[0].dropna(how='all')
data.head()

,INSP-DATE,REC-DATE,ENTRYDATE,NAME,Int/Ext,RESULTS,NYSCERTIFICATE
0,10/17/2016,10/27/2016,01/04/2017,VAZQUEZ EDWIN,NaN,NO DEFECTS,P 001526
1,06/26/2015,08/05/2015,08/19/2015,VAZQUEZ EDWIN,NaN,NO DEFECTS,P 001526
2,09/17/2010,09/22/2010,12/17/2014,VAZQUEZ EDWIN,NaN,NO DEFECTS,P 1526
3,08/08/2014,09/05/2014,09/15/2014,VAZQUEZ EDWIN,NaN,NO DEFECTS,P 001526
4,10/08/2013,10/31/2013,10/31/2013,VAZQUEZ EDWIN,NaN,NO DEFECTS,P 001526


### Scrape all boilers

In [67]:
df = pd.DataFrame(columns=['boiler_id'])

In [68]:
df.boiler_id.nunique()

46220

In [80]:
start_time = time()
req = 0

#Skip already downloaded ones (when re-running after error)
mask = ~boilers['Boiler ID'].isin(df.boiler_id)
total = sum(mask)
error = False

for i, boil in boilers[mask].iterrows():
# To run simultaneously, change this
#     if boil.short_id%2 == 0:
#         continue
    
    url1 = 'http://a810-bisweb.nyc.gov/bisweb/BoilerComplianceQueryServlet?allborough={}&boilernumber={}'.format(
        boil.boro,
        boil.short_id)
    
    while True:
        
        response1 = requests.get(url1,
                                 proxies={"http": "http://{}:{}@us-wa.proxymesh.com:31280".format(ProxyMeshUser,ProxyMeshPass)})
            
        if response1.status_code != 200 or 'Bad Request' in response1.text:
            print('Bad Request')
            continue
            
        html1 = BeautifulSoup(response1.text, 'html.parser')
        
        if 'Access Denied' in str(html1.find('title')):
            print('Access Denied')
            continue
        elif 'Visitor Prioritization' not in str(html1.find('title')):
            break

        print('Visitor priorization')

    boiler_list = html1.find_all('table')[3]
    boiler_list_df = pd.read_html(str(boiler_list), flavor='bs4', header=0)[0].dropna(how='all')
    
    try:
        idx = boiler_list_df.loc[(boiler_list_df['SER#'] == boil['serial'])\
                                 & (boiler_list_df['STATUS'] != 'VOID')].index[0]

        #Get link for second request
        link = boiler_list.find_all('a')[idx]
        url2 = 'http://a810-bisweb.nyc.gov/bisweb/' + link.get('href')

        while True:
            
            response2 = requests.get(url2,
                                     proxies={"http": "http://{}:{}@us-wa.proxymesh.com:31280".format(ProxyMeshUser,ProxyMeshPass)})
                
            if response2.status_code != 200 or 'Bad Request' in response2.text:
                print('Bad Request')
                continue
            
            html2 = BeautifulSoup(response2.text, 'html.parser')

            if 'Access Denied' in str(html2.find('title')):
                print('Access Denied')
                continue
                
            elif 'Visitor Prioritization' not in str(html2.find('title')):
                break

            print('Visitor priorization')


        #Get table
        table = html2.find_all('table')[6]
        data = pd.read_html(str(table), flavor='bs4', skiprows=4, header=0)[0].dropna(how='all')

        #Add Boiler ID and Bin Number
        data['bin_number'] = boil['Bin Number']
        data['boiler_id'] = boil['Boiler ID']

        df = df.append(data)
    except (IndexError, KeyError):
        df = df.append(pd.Series(data=[boil['Boiler ID'], 'NOT FOUND'], index=['boiler_id', 'Int/Ext']), ignore_index=True)
        print('Active boiler record not found')
    finally:
        req += 1
        
        current_time = time()
        elapsed_time = current_time - start_time
        print('Boiler {} of {}; Frequency: {:.2f} requests/s'.format(req, total,req/elapsed_time))
        clear_output(wait = True)

KeyboardInterrupt: 

In [79]:
df.to_csv('data/boiler_temp_3.csv', index=False)

In [81]:
df_final = pd.concat([pd.read_csv('data/boiler_temp_3.csv'),
                      pd.read_csv('data/boiler_temp_4.csv'),
                      pd.read_csv('data/boiler_temp_5.csv'),
                      pd.read_csv('data/boiler_temp_6.csv')])
df_final.drop_duplicates(inplace=True)
df_final.head()

,ENTRYDATE,INSP-DATE,Int/Ext,NAME,NYSCERTIFICATE,REC-DATE,RESULTS,bin_number,boiler_id
0,05/31/2016,10/23/2015,NaN,TRAVELERS INSURANCE CO,3012,10/30/2015,DEFECTS CORRECTED,1059945.0,10000008199Y0001
1,03/10/2016,02/04/2016,NaN,THE HARTFORD STEAM BLR INSP INS,5240,03/08/2016,NO DEFECTS,1059945.0,10000008199Y0001
2,09/22/2015,07/21/2015,NaN,TRAVELERS INSURANCE CO,5499,08/13/2015,DEFECTS EXIST,1059945.0,10000008199Y0001
3,12/24/2014,09/18/2014,NaN,TRAVELERS INSURANCE CO,3012,10/01/2014,NO DEFECTS,1059945.0,10000008199Y0001
4,04/11/2013,03/13/2013,NaN,TRAVELERS INSURANCE CO,3012,03/27/2013,NO DEFECTS,1059945.0,10000008199Y0001


In [82]:
df_final.boiler_id.nunique()

95708

In [84]:
(~boilers['Boiler ID'].isin(df_final.boiler_id)).sum()

0

In [85]:
df_final.to_csv('data/bis_boiler_final.csv', index=False)

### Done!